In [1]:
import sys

sys.path.insert(0, '..')

## Setup the model area

In [2]:
from morpheus.modflow.types.discretization import SpatialDiscretization
from morpheus.modflow.types.discretization.spatial import Rotation, LengthUnit, Grid, GridCells, Crs
from morpheus.modflow.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(0)
length_unit = LengthUnit.meters()
x_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
y_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  x_coordinates=x_coordinates,
  y_coordinates=y_coordinates,
  rotation=rotation,
)

spatial_discretization = SpatialDiscretization(
  geometry=polygon,
  grid=grid,
  affected_cells=GridCells.from_polygon(polygon=polygon, grid=grid),
  crs=Crs.from_str('EPSG:4326')
)

spatial_discretization


SpatialDiscretization(geometry=Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96629040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon'), grid=Grid(origin=Point(coordinates=(13.922224410263878, 50.9662904037036), type='Point'), x_distances=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], y_distances=[27.74944856585935, 27.74944856585935, 27.749448565859346, 27.749448565859353, 27.74944856585934, 27.749448565859353, 27.749448565859325, 27.74944856585938, 27.749448565859325, 27.749448565859353], rotation=Rotation(value=0), length_unit=LengthUnit(unit=2)), affected_cells=GridCells(shape=(10, 10), data=[GridCell(x=0, y=0, value=True), GridCell(x=0, y=1, value=True), GridCell(x=0, y=2, value=True), GridCell(x=0, y=

## Setup Time Discretization

In [3]:
from morpheus.modflow.types.discretization.time.TimeUnit import TimeUnit
from morpheus.modflow.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState
from morpheus.modflow.types.discretization.time import TimeDiscretization
from datetime import datetime

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 2, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 3, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 4, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 5, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 6, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 7, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 8, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 9, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 10, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 11, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 12, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.no()
    ),
  ]),
  time_unit=TimeUnit.days()
)

time_discretization

TimeDiscretization(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0)), end_date_time=StartDateTime(value=datetime.datetime(2020, 12, 31, 0, 0)), stress_periods=StressPeriodCollection(values=[StressPeriod(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0)), number_of_time_steps=NumberOfTimeSteps(value=1), time_step_multiplier=TimeStepMultiplier(value=1), steady_state=IsSteadyState(value=True)), StressPeriod(start_date_time=StartDateTime(value=datetime.datetime(2020, 2, 1, 0, 0)), number_of_time_steps=NumberOfTimeSteps(value=1), time_step_multiplier=TimeStepMultiplier(value=1), steady_state=IsSteadyState(value=False)), StressPeriod(start_date_time=StartDateTime(value=datetime.datetime(2020, 3, 1, 0, 0)), number_of_time_steps=NumberOfTimeSteps(value=1), time_step_multiplier=TimeStepMultiplier(value=1), steady_state=IsSteadyState(value=False)), StressPeriod(start_date_time=StartDateTime(value=datetime.datetime(2020, 4, 1, 0, 0)), number_of_time_st

In [4]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))

m.fit_bounds(m.get_bounds())
m

## Setup modflow model

In [5]:
from morpheus.modflow.types.ModflowModel import ModflowModel

modflow_model = ModflowModel.new()
modflow_model = modflow_model.with_updated_time_discretization(time_discretization)
modflow_model = modflow_model.with_updated_spatial_discretization(spatial_discretization=spatial_discretization)

# Setup well boundary

In [6]:
from morpheus.modflow.types.geometry import Point
from morpheus.modflow.types.boundaries.Boundary import BoundaryName, WellBoundary
from morpheus.modflow.types.boundaries.WellObservation import WellRawDataItem, PumpingRate

layers_list = [modflow_model.soil_model.layers[0].id]
well_boundary = WellBoundary.from_geometry(
  name=BoundaryName('well_boundary'),
  geometry=Point((13.923, 50.966)),
  grid=grid,
  affected_layers=[layers_list[0]],
  raw_data=[
    WellRawDataItem(
      date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
      pumping_rate=PumpingRate.from_float(-5000),
    ),
    WellRawDataItem(
      date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
      pumping_rate=PumpingRate.from_float(-1000),
    ),
  ]
)

well_boundary.affected_cells = well_boundary.affected_cells.filter(
  lambda cell: spatial_discretization.affected_cells.contains(cell))

m.add_child(folium.GeoJson(well_boundary.as_geojson()))
m.add_child(folium.GeoJson(well_boundary.affected_cells.as_geojson(grid).as_geojson()))
for well_boundary.observation in well_boundary.observations:
  m.add_child(folium.GeoJson(well_boundary.observation.as_geojson()))
m.fit_bounds(m.get_bounds())
m

## Calculate constant head stress periods

In [7]:
from morpheus.modflow.infrastructure.calculation.modflow_2005.packages.WelPackageMapper import \
  calculate_wel_boundary_stress_period_data

result = calculate_wel_boundary_stress_period_data(
  spatial_discretization=spatial_discretization,
  time_discretization=time_discretization,
  soil_model=modflow_model.soil_model,
  wel_boundary=well_boundary,
)
result.to_dict()

{0: [[0, 1, 2, -3610.0456621004564]],
 1: [[0, 1, 2, -3500.4566210045664]],
 2: [[0, 1, 2, -3390.867579908676]],
 3: [[0, 1, 2, -3279.452054794521]],
 4: [[0, 1, 2, -3168.0365296803648]],
 5: [[0, 1, 2, -3056.6210045662106]],
 6: [[0, 1, 2, -2945.2054794520545]],
 7: [[0, 1, 2, -2831.9634703196343]],
 8: [[0, 1, 2, -2720.5479452054797]],
 9: [[0, 1, 2, -2609.132420091324]],
 10: [[0, 1, 2, -2497.7168949771694]],
 11: [[0, 1, 2, -2388.1278538812785]]}